# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **
The answer is Classification.
The goal of the project is to identify students who might need early intervention before they fail to graduate. Thus, the label of the project is either yes student needs an early intervention, or no student does not need an early intervention. For this reason, this project is a classificaiotn problem and sepcifically a binary classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [11]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [12]:
# TODO: Calculate number of students
# Corresponds to the number of rows
n_students = student_data.shape[0]

# TODO: Calculate number of features
# Corresponds to the number of columns
n_features = student_data.shape[1] - 1 # Subtract one column that corresponds to the label.

# TODO: Calculate passing students
n_passed = np.where(student_data['passed'] == 'yes')[0].size

# TODO: Calculate failing students
# !!!!!!! Better simpler method?!
n_failed = np.where(student_data['passed'] == 'no')[0].size

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/n_students*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [13]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:" 
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [14]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output
X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


In [15]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
# X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, test_size = 95, random_state = 42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **
The seven learning models could be categorized either as parametric or nonparametric.
Parametric classifiers included Linear Gaussian Naive Bayes, Stochastic Gradient Descent, and Logistic Regression. Nonparametric was composed of the remaining classifiers: Decision Trees, Ensemble Methods, K-Nearest Neighbors, and Support Vector Machine.

The number of values, features, and characteristics of the student data were examined to determin the category of classifiers to be used. First, data size was concluded to be small based on [skcikit learn algorighm cheat sheet](http://scikit-learn.org/stable/tutorial/machine_learning_map/), which seemed to set 100k as a boundary between large and small data numbers. Second, features of student data were relatively small in number (compared to the size of samples) and not so complex. Given the two conditions, relatively small-sized and not overly complex data, **parametric classifier**[<sup>reference</sup>](http://machinelearningmastery.com/parametric-and-nonparametric-machine-learning-algorithms/) was chosen.
However, inspecting the details of feature listed in README.md file suggested that many features are somewhat interconnected (e.g. workday alcohol consumption(Dalc) and weekend alcohol consumption(Walc)). Considering that, Gaussian Naive Bayes was not selected, which works under the assumption that features are idependent. Instead, Linear Suppor Vector Machine was chosen based on the [scikit-learn algorithm cheatsheet](http://scikit-learn.org/stable/tutorial/machine_learning_map/).

**Classifiers**  
1. Stochastic Gradient Descent
    * Real-world application: [User recommendation](https://books.google.com/books?id=H_WbBQAAQBAJ&pg=PA219&lpg=PA219&dq=kaggle+stochastic+gradient+descent&source=bl&ots=7JVY0hpeAL&sig=LGzrURzOYUugXEhA5Itwk_RErcU&hl=ko&sa=X&ved=0ahUKEwjcnYeH1d7QAhVbHGMKHXaBBJcQ6AEIXDAH#v=onepage&q=kaggle%20stochastic%20gradient%20descent&f=false)
    * Strengths: Cheap and fast learning.
    * Weaknesses: Takes baby steps (small gradients at a time) to make final classification, thus unstable in nature.
    * Why a good candidate?: Does a great job in fast and simple steps finding the local minima.
    * Reference: 
        - (www.csie.ntu.edu)[https://www.csie.ntu.edu.tw/~htlin/course/ml13fall/doc/11_handout.pdf]
        - [Stanford Lecture Note](https://datajobs.com/data-science-repo/Generalized-Linear-Models-[Andrew-Ng].pdf)
        

2. Logistic Regression
    * Real-world application: [Data Mining](http://repository.cmu.edu/cgi/viewcontent.cgi?article=1217&context=robotics)
    * Strengths: Applied a simple linear regression for binary classification that results in probability scores for observations.
    * Weaknesses: 
        - Doesn’t perform well when feature space is too large
        - Doesn’t handle large number of categorical features/variables well
    * Why a good candidate?: Student data require binary classification and are relatively small with small number of features.
    * Reference: 
        - [blog.echen.me](http://blog.echen.me/2011/04/27/choosing-a-machine-learning-classifier/)
        - [ufldl.stanford.edu](http://ufldl.stanford.edu/tutorial/supervised/LogisticRegression/)
        - [Stanford Lecture Note](https://datajobs.com/data-science-repo/Generalized-Linear-Models-[Andrew-Ng].pdf)
    
    
3. Linear Support Vector Machine
    * Real-world application: [In Geo and Environmental Sciences](http://www.clopinet.com/SVM.applications.html)
    * Strengths: Memory Efficient, effective in cases where number of dimensions is greater than the number of samples, and a number of different kernel functions can be specified.
    * Weaknesses: Does not work well if the number of features is far greater than the number of samples, and does not directly return probability estimates.
    * Why a good candidate?: This option was chosen based on its popularity and Scikit Learn recommendation. Taking a look at the sizes of features compared to that of samples of the student data ensured that they will not be hit by the first weakness mentioned above. Moreover, if the assumption that many features of student data are interdependent holds true, data transformation by applying Support Vector Machine may lead to good predictions.
        * Reference: [Scikit-Learn](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [16]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [17]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

clf_A = SGDClassifier(random_state=42)
clf_B = LogisticRegression(random_state=42)
clf_C = SVC(kernel='linear', random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]

X_train_300 = X_train[0:300]
y_train_300 = y_train[0:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# REGEX method of doing this loop?
X_train_ = [X_train_100, X_train_200, X_train_300]
y_train_ = [y_train_100, y_train_200, y_train_300]

for i in range (0,3):
    X_train = X_train_[i]
    y_train = y_train_[i]
    train_predict(clf_A, X_train, y_train, X_test, y_test)
    train_predict(clf_B, X_train, y_train, X_test, y_test)
    train_predict(clf_C, X_train, y_train, X_test, y_test)


Training a SGDClassifier using a training set size of 100. . .
Trained model in 0.0677 seconds
Made predictions in 0.0077 seconds.
F1 score for training set: 0.8075.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.8025.
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0030 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8671.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7068.
Training a SVC using a training set size of 100. . .
Trained model in 0.0117 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8750.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.6614.
Training a SGDClassifier using a training set size of 200. . .
Trained model in 0.0008 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 0.8212.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7703.
Training a LogisticRegression using a training set size of 200. . .
Train

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Stochatic Gradient Descent (SGD)**  

| Training Set Size | Training Time [sec] | Prediction Time (test) [sec]  | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0014            |        0.0006          |   0.0004         |  0.0645         |
| 200               |  0.0017                 |     0.0006             | 0.7681           | 0.6552          |
| 300               | 0.0014                  |   0.0006               |0.8262            |  0.7919         |

** Classifer 2 - Logistic Regression **  

| Training Set Size | Training Time [sec] | Prediction Time (test) [sec] | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0019                 |      0.0003            |     0.8593       |   0.7647        |
| 200               |   0.0028                |    0.0003              | 0.8562           |0.7914           |
| 300               |  0.0061                 |  0.0006                |  0.8468          |   0.8060        |

** Classifer 3 - Support Vector Machine (SVM)**  

| Training Set Size | Training Time [sec] | Prediction Time (test) [sec] | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0027                  |     0.0093             |   0.8777         |  0.7746         |
| 200               | 0.0055                  | 0.0020                 | 0.8679           |  0.7815         |
| 300               |0.0089                   |  0.0025                |  0.8761          |  0.7838         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

**Logistic Regression** will be the best classifier, considering all constraints including available data, limited resources, cost, and performance. As F<sub>1</sub> scores indicate, all three models performs well in predicting new data, as more training data are given. However, training time for both Logistic Regression and Support Vector Machine(SVM) classifiers increased with as more training data were given. For example, when training set size increased from 100 to 300, training time for both Logistic Regression classifier and SVM classifier tripled. Training time is where we can greatly benefit from Stochastic Gradient Descent(SGD) classifier; it only calculates the local gradient for finding local minima. 

Considering cost and performance, SGD classifier will be the best option. However, as can be seen from the table, SGD classifier performed the worst at smaller number training set size. At training set size of 100, SGD classifier returned F<sub>1</sub> score of 0.0004 for traing and 0.0645 for testing. In order to make good predictions at conditions with very limited resources (number of resources under 300), a better alternative should be used. Thus, I concluded that **Logistic Regression** is the best classifier. (If we are to have training size greater than 300, SGD will be the best option.)

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** 

Logistic Regression Classifier is a _classifier_ that applies a _logistic_ function to a linear _regression_ model to do classification. 

1. Applying Logistic Regression to the training data results in weights for each feature. These weights indicate how much each feature contributes to the correct (with least error) binary classification.
2. When new data is introduced, feature values are multiplied by their feature weights and summed up. 
3. Applying the result to a _logistic_ function outputs a result between 0 and 1. This number corresponds to the probability of the new data being classified as positive of the binary label.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [18]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1.0, 10, 100], 'random_state': [13, 29, 42, 53, 71]}

# TODO: Initialize the classifier 
clf = LogisticRegression()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))


Made predictions in 0.0002 seconds.
Tuned model has a training F1 score of 0.8291.
Made predictions in 0.0003 seconds.
Tuned model has a testing F1 score of 0.7891.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

Logistic Regression model's F<sub>1</sub> score is 0.7891. By default GridSearchCV conducts 3-fold cross validation, which leaves us with a training set size of 200. Thus the resulting F<sub>1</sub> score of 0.7891 agrees well with the trend from the table above where we had a F<sub>1</sub> very close to 0.7891 (which is 0.7914) for the training set size of 200. This also means that the tuned Logistic Regreession model does not perform any better than the untuned model. 
This can be due to the tuning parameters used introducing negligible difference to the model. The parameters 'penalty' and 'C' are used to solve overfitting. Thus, it is likely that the model was not prone to overfitting even when it was trained with default parameters. 'Random_state', which shuffles the data randomly, also did not introduce noticable difference because, possibly, the testing data was not complex enough for shuffling to make significant improvements.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.